In [ ]:
import numpy as np
import cv2
import os
import pandas as pd
import tensorflow as tf
import random
import copy
import matplotlib.pyplot as plt
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from google.colab import drive

drive.mount('/content/drive/')

## Folder path for the UTK image dataset

folder_path = '/content/drive/MyDrive/data/crop_part1/crop_part1/'

## Paths for the npy files created for ease(change to wherever your .npy files
## are stored)

images_path = '/content/drive/MyDrive/data/images.npy'

ages_path = '/content/drive/MyDrive/data/ages.npy'

genders_path = '/content/drive/MyDrive/data/genders.npy'

races_path = '/content/drive/MyDrive/data/races.npy'

dates_path = '/content/drive/MyDrive/data/dates.npy'

def load_images_from_folder(folder):
    images = []
    ages = []
    genders = []
    races = []
    dates = []
    for filename in os.listdir(folder):
        print(filename)
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
          if (len(filename.split('_')) == 4):
            age, gender, race, date = filename_data(filename)
            images.append(img)
            ages.append(age)
            genders.append(gender)
            races.append(race)
            dates.append(date)
    return images, ages, genders, races, dates


def filename_data(filename):
  img_name = filename
  data = img_name.split('_')
  age = data[0]
  gender = data[1]
  race = data[2]
  data2 = data[3]
  date = data2.split('.')[0]
  return age, gender, race, date
def get_accuracy(predicted, actual):
  n = len(predicted)
  tot = 0
  for i in range(n):
    if predicted[i] == actual[i]:
      tot = tot + 1
  return tot/n

In [ ]:
## This is just to save the data for later so loading isn't required every time
## Don't run if you already have the data in the .npy files
images, ages, genders, races, dates = load_images_from_folder(folder_path)
np.save('images.npy', images)
np.save('ages.npy', ages)
np.save('genders.npy', genders)
np.save('races.npy', races)
np.save('dates.npy', dates)

In [ ]:
## Loading
images = np.load(images_path)
ages = np.load(ages_path)
genders = np.load(genders_path)
races = np.load(races_path)
dates = np.load(dates_path)

('0', '1', '0', '20170110183726390')

In [ ]:
## gender: 0=male, 1=female
## race: 0=white, 1=black, 2=asian, 3=indian

genders = genders.astype(np.uint8)
images.shape
df = pd.DataFrame({
    'Age': ages,
    'Gender': genders,
    'Race': races,
    'Date': dates
})
white = df.Race == '0'
black = df.Race == '1'
asian = df.Race == '2'
indian = df.Race == '3'

white_images = images[white]
white_genders = genders[white]
black_images = images[black]
black_genders = genders[black]
asian_images = images[asian]
asian_genders = genders[asian]
indian_images = images[indian]
indian_genders = genders[indian]

n_white = len(white_genders)
n_black = len(black_genders)
n_asian = len(asian_genders)
n_indian = len(indian_genders)
n = min(n_white,n_black,n_asian,n_indian)
random.seed(100)
white_idx = random.sample(range(n_white), n)
asian_idx = random.sample(range(n_asian), n)
indian_idx = random.sample(range(n_indian), n)

white_images_2 = white_images[white_idx]
white_genders_2 = white_genders[white_idx]
asian_images_2 = asian_images[asian_idx]
asain_genders_2 = asian_genders[asian_idx]
indian_images_2 = indian_images[indian_idx]
indian_genders_2 = indian_genders[indian_idx]

NameError: ignored

**MODEL**

In [ ]:
## This is the model I made(also I'm not even sure if it's right but I tried)
def createModel():
  model = tf.keras.models.Sequential()

  model.add(
    layers.Conv2D(64, (5,5),
                  activation = 'relu',
                  input_shape = (200,200,3))
  )
  model.add(
    layers.MaxPooling2D((2,2))
  )
  model.add(
    layers.Conv2D(128, (3,3),
                  activation = 'relu')
  )
  model.add(
    layers.MaxPooling2D((2,2))
  )
  model.add(
    layers.Flatten()
  )
  model.add(
    layers.Dense(128, activation = 'relu')
  )
  model.add(
      layers.Dropout(.5)
  )
  model.add(
      layers.Dense(128, activation = 'relu')
  )
## Sigmoid for binary classification
  model.add(
    layers.Dense(1, activation = 'sigmoid')
  )
## Might need to change learning rate later
  model.compile(
  optimizer = optimizers.SGD(learning_rate=1e-6),
  loss = 'binary_crossentropy',
  metrics = ['accuracy']
  )
  return model

In [ ]:
white_model = createModel()
black_model = createModel()
asian_model = createModel()
indian_model = createModel()

In [ ]:
history_white = white_model.fit(white_images_2, white_genders_2, epochs = 50, batch_size = 60)
history_black = black_model.fit(black_images, black_genders, epochs = 50, batch_size = 60)
history_asian = asian_model.fit(asian_images_2, asian_genders_2, epochs = 50, batch_size = 60)
history_indian = indian_model.fit(indian_images_2, indian_genders_2, epochs = 50, batch_size = 60)

4626

In [ ]:
w_p_w = (white_model.predict(white_images_2) > 0.5).astype("int32")
w_p_b = (white_model.predict(black_images) > 0.5).astype("int32")
w_p_a = (white_model.predict(asian_images_2) > 0.5).astype("int32")
w_p_i = (white_model.predict(indian_images_2) > 0.5).astype("int32")

w = get_accuracy(w_p_w, white_genders_2)
b = get_accuracy(w_p_b, black_genders)
a = get_accuracy(w_p_a, asian_genders_2)
i = get_accuracy(w_p_i, indian_genders_2)
fig = plt.figure()
labs = ['white', 'black', 'asian', 'indian']
accuracies = [w,b,a,i]
plt.bar(labs,accuracies)
plt.ylabel('accuracies')
plt.title('Prediction Accuracies of Model trained only on white faces')
plt.show()

In [ ]:
b_p_w = (black_model.predict(white_images_2) > 0.5).astype("int32")
b_p_b = (black_model.predict(black_images) > 0.5).astype("int32")
b_p_a = (black_model.predict(asian_images_2) > 0.5).astype("int32")
b_p_i = (black_model.predict(indian_images_2) > 0.5).astype("int32")

w = get_accuracy(b_p_w, white_genders_2)
b = get_accuracy(b_p_b, black_genders)
a = get_accuracy(b_p_a, asian_genders_2)
i = get_accuracy(b_p_i, indian_genders_2)
fig = plt.figure()
labs = ['white', 'black', 'asian', 'indian']
accuracies = [w,b,a,i]
plt.bar(labs,accuracies)
plt.ylabel('accuracies')
plt.title('Prediction Accuracies of Model trained only on Black faces')
plt.show()

In [ ]:
a_p_w = (asian_model.predict(white_images_2) > 0.5).astype("int32")
a_p_b = (asian_model.predict(black_images) > 0.5).astype("int32")
a_p_a = (asian_model.predict(asian_images_2) > 0.5).astype("int32")
a_p_i = (asian_model.predict(indian_images_2) > 0.5).astype("int32")

w = get_accuracy(a_p_w, white_genders_2)
b = get_accuracy(a_p_b, black_genders)
a = get_accuracy(a_p_a, asian_genders_2)
i = get_accuracy(a_p_i, indian_genders_2)
fig = plt.figure()
labs = ['white', 'black', 'asian', 'indian']
accuracies = [w,b,a,i]
plt.bar(labs,accuracies)
plt.ylabel('accuracies')
plt.title('Prediction Accuracies of Model trained only on Asian faces')
plt.show()

In [ ]:
i_p_w = (indian_model.predict(white_images_2) > 0.5).astype("int32")
i_p_b = (indian_model.predict(black_images) > 0.5).astype("int32")
i_p_a = (indian_model.predict(asian_images_2) > 0.5).astype("int32")
i_p_i = (indian_model.predict(indian_images_2) > 0.5).astype("int32")

w = get_accuracy(i_p_w, white_genders_2)
b = get_accuracy(i_p_b, black_genders)
a = get_accuracy(i_p_a, asian_genders_2)
i = get_accuracy(i_p_i, indian_genders_2)
fig = plt.figure()
labs = ['white', 'black', 'asian', 'indian']
accuracies = [w,b,a,i]
plt.bar(labs,accuracies)
plt.ylabel('accuracies')
plt.title('Prediction Accuracies of Model trained only on Indian faces')
plt.show()